In [2]:
import cv2
import numpy as np
import keras

## Class of Classifier

In [17]:
class LinearClassifier():
    def __init__(self, Rogue, Hover):
        self.epochs = 100
        self.hover = self.preprocess_data(Hover)
        self.rogue = self.preprocess_data(Rogue)
        
        self.model = model = keras.models.Sequential([
            keras.layers.InputLayer((80,80,3)),
            keras.layers.Conv2D(64, 3, padding= 'same'),
            keras.layers.MaxPooling2D(),
            keras.layers.Flatten(),
            keras.layers.Dense(1, activation= 'sigmoid')
        ])
    
    @staticmethod
    def preprocess_data(image):
        image_toresize = image.copy()
        height, width = image_toresize.shape[:2]
        blank_image = np.zeros((80, 80, 3), np.uint8)
        blank_image[:,:] = (255,255,255)
        l_img = blank_image.copy()
        x_offset = 40 - int(width/2)
        y_offset = 40 - int(height/2)
        l_img[y_offset:y_offset+height, x_offset:x_offset+width] = image_toresize.copy()
        return np.array(l_img[np.newaxis, :, :, :])
    
    def prep_data(self):
        _hover = self.hover
        _rogue = self.rogue
        X_train = np.vstack([_hover, _rogue])
        y_train = np.array([1.0,0.0])
        return X_train, y_train
        
    def train_classifier(self):
        X_train, y_train = self.prep_data()
        self.model.compile(optimizer= keras.optimizers.SGD(lr=0.01, momentum=0.9, clipnorm=1.0), loss= keras.losses.binary_crossentropy)
        self.model.fit(X_train, y_train, epochs= self.epochs, verbose=1)
        return self.model

In [18]:
LC = LinearClassifier(Rogue= Rogue, Hover= hoverTank_1)
model = LC.train_classifier()
# army = ArmyManagement(army_m_window= armyManagement_cut, unit=hoverTank_1, classifier= LC)

Epoch 1/100
2/2 [==============================] - 0s 57ms/step - loss: 36.1983
Epoch 2/100
2/2 [==============================] - 0s 9ms/step - loss: 44.9525
Epoch 3/100
2/2 [==============================] - 0s 11ms/step - loss: 34.2486
Epoch 4/100
2/2 [==============================] - 0s 8ms/step - loss: 48.0275
Epoch 5/100
2/2 [==============================] - 0s 14ms/step - loss: 42.9568
Epoch 6/100
2/2 [==============================] - 0s 9ms/step - loss: 19.7518
Epoch 7/100
2/2 [==============================] - 0s 11ms/step - loss: 7.8188
Epoch 8/100
2/2 [==============================] - 0s 8ms/step - loss: 42.3947
Epoch 9/100
2/2 [==============================] - 0s 12ms/step - loss: 34.0238
Epoch 10/100
2/2 [==============================] - 0s 12ms/step - loss: 1.9684e-09
Epoch 11/100
2/2 [==============================] - 0s 11ms/step - loss: 66.8336
Epoch 12/100
2/2 [==============================] - 0s 10ms/step - loss: 46.4095
Epoch 13/100
2/2 [=====================

Epoch 100/100
2/2 [==============================] - 0s 8ms/step - loss: 0.0000e+00


# Class for army management

In [14]:
class ArmyManagement():
    def __init__(self, army_m_window, unit, classifier, threshold= 0.8):
        self.army_m_window = army_m_window
        self.unit = unit
        self.classifier = classifier
        self.model = classifier.train_classifier()
        
        self.found_units = []
        self.blank = army_m_window.copy() * 0
        
        self.result_blank = army_m_window.copy() * 0
        self.result = army_m_window.copy()
        
        self.threshold = threshold
        self.n_found_units = 0
        
        self.counter = 0
        #self.template_finder_replace()
    
    def process_contours(self, cnt):
        x_min = min(cnt[:, 0][:, 0])
        x_max = max(cnt[:, 0][:, 0])
        y_min = min(cnt[:, 0][:, 1])
        y_max = max(cnt[:, 0][:, 1])
        result = self.army_m_window[y_min:y_max, x_min:x_max]
        result = self.classifier.preprocess_data(result)
        self.process_found_units(result)
    
    def process_found_units(self, unit_):
        class_ = int(self.model.predict(unit_))
        self.found_units.append(unit_)
        self.counter =+ 1
        path = f'./MasterOpenCV/FoE/Army/Image_{class_}_{self.counter}.png'
        cv2.imwrite(path, unit_[0])

In [15]:
armyManagement = cv2.imread('./MasterOpenCV/FoE/Army/ArmyManagement.png')
armyManagement_cut = cv2.imread('./MasterOpenCV/FoE/Army/ArmyManagement_cut.png')
armyManagement_hurt = cv2.imread('./MasterOpenCV/FoE/Army/ArmyManagement_HurtUnits.png')

hoverTank_1 = cv2.imread('./MasterOpenCV/FoE/Army/HoverTank.png')
hoverTank_2 = cv2.imread('./MasterOpenCV/FoE/Army/HoverTank_2.png')
Rogue = cv2.imread('./MasterOpenCV/FoE/Army/Rogue.png')

In [19]:
img = armyManagement_cut[40:-10, 10:280]
sizeX, sizeY = img.shape[1], img.shape[0]

nRows = 2
mCols = 4
for i in range(0,nRows):
    for j in range(0, mCols):
        roi = img[i*int(sizeY/nRows):i*int(sizeY/nRows) + int(sizeY/nRows),
                  j*int(sizeX/mCols):j*int(sizeX/mCols) + int(sizeX/mCols)]
        cv2.imwrite('./MasterOpenCV/FoE/Army/FoundUnits/patch_'+str(i)+str(j)+".jpg", roi)

(144, 270, 3)


In [20]:
hover_segment = cv2.imread('./MasterOpenCV/FoE/Army/FoundUnits/patch_00.jpg')
rogue_segment = cv2.imread('./MasterOpenCV/FoE/Army/FoundUnits/patch_01.jpg')

hover_segment = LC.preprocess_data(hover_segment)
rogue_segment = LC.preprocess_data(rogue_segment)

hoverTank_1_ = LC.preprocess_data(hoverTank_1)
hoverTank_2_ = LC.preprocess_data(hoverTank_2)
rogue_ = LC.preprocess_data(Rogue)

print(LC.model.predict(hover_segment), LC.model.predict(rogue_segment))
print(LC.model.predict(hoverTank_1_), LC.model.predict(rogue_))

[[1.]] [[0.]]
[[1.]] [[0.]]


## Driver Code

In [ ]:
armyManagement = cv2.imread('./MasterOpenCV/FoE/Army/ArmyManagement.png')
armyManagement_hurt = cv2.imread('./MasterOpenCV/FoE/Army/ArmyManagement_HurtUnits.png')

hoverTank_1 = cv2.imread('./MasterOpenCV/FoE/Army/HoverTank.png')
hoverTank_2 = cv2.imread('./MasterOpenCV/FoE/Army/HoverTank_2.png')
Rogue = cv2.imread('./MasterOpenCV/FoE/Army/Rogue.png')

# cv2.imshow('Army Management', armyManagement)
# cv2.waitKey()
# cv2.imshow('Army Management', hoverTank_1)
# cv2.waitKey()
# cv2.imshow('Army Management', hoverTank_2)
# cv2.waitKey()
# cv2.destroyAllWindows()

# nRogues, rogueCount = template_finder(armyManagement_hurt, Rogue, threshold= 0.9, count= True)
# nHovers, hoverCount = template_finder(armyManagement_hurt, hoverTank_1, threshold= 0.8, count= True)

nRogues = template_finder(armyManagement_hurt, hoverTank_2, threshold= 0.8, count= True)
cv2.destroyAllWindows()


# cv2.imshow('Army Management', nRogues)
# cv2.waitKey()
# cv2.destroyAllWindows()

# hoverCount

## Old Project

In [ ]:
mainCity = cv2.imread('./MasterOpenCV/FoE/Something.png')
aiding = cv2.imread('./MasterOpenCV/FoE/Aid.png')
FT = cv2.imread('./MasterOpenCV/FoE/FT.png')
GB = cv2.imread('./MasterOpenCV/FoE/GB.png')
MPed = cv2.imread('./MasterOpenCV/FoE/Motivated.png')
barrack = cv2.imread('./MasterOpenCV/FoE/Barrack.png')

(toBeAided, noAid) = template_finder(mainCity, aiding, count=True)
print(noAid)
friendTavern = template_finder(mainCity, FT)
GB = template_finder(mainCity, GB)
Motivated = template_finder(mainCity, MPed, threshold=0.5)
Troops = template_finder(mainCity, barrack)

cv2.imshow('Ready troops', Troops)
cv2.waitKey()
cv2.imshow('Aiding', toBeAided)
cv2.waitKey()
cv2.imshow('Friend Tavern', friendTavern)
cv2.waitKey()
cv2.imshow('Great Buildings', GB)
cv2.waitKey()
cv2.imshow('Motivated', Motivated)
cv2.waitKey()

cv2.destroyAllWindows()